In [1]:
'''General Libraries'''
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 500)

'''Statistic'''
import scipy
from scipy import stats
from scipy.stats import norm
import datetime

'''Scikit Learn'''
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV

'''Ploting Libraries'''
from plotly.offline import iplot, plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = "notebook" 


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_palette('RdBu')

'''Miscellaneous'''
from yellowbrick.regressor import ResidualsPlot
import missingno as msno

'''Seeds'''
import random
random.seed(10)
np.random.seed(11)


# Data Overview

In [2]:
df = pd.read_csv('2014_clean.csv')

In [3]:
df.head(8)

,name,5k,10k,20k,half,25k,30k,35k,40k,official,pace,rank_overall,rank_in_gender_only,rank_in_division,age,gender,country,state,city,bib
0,"Yamamoto, Hiroyuki",8.02,17.37,37.65,39.72,47.67,59.18,71.40,80.43,85.25,3.27,8,8,8,47,M,JPN,others,Fukuoka,W1
1,"Jeptoo, Rita",16.22,32.58,65.83,69.47,82.43,99.33,116.37,132.10,138.95,5.30,21,1,1,33,F,KEN,others,Eldoret,F1
2,"Van Dyk, Ernst F.",7.75,16.62,36.10,38.03,45.80,56.45,67.42,76.10,80.60,3.08,1,1,1,41,M,RSA,others,Paarl,W2
3,"Dibaba, Mare",16.20,32.57,65.83,69.47,82.43,99.33,116.37,132.95,140.58,5.37,27,3,3,24,F,ETH,others,Shoa,F2
4,"Hokinoue, Kota",8.02,17.12,36.58,38.60,46.37,57.03,67.83,76.72,81.23,3.10,2,2,2,40,M,JPN,others,Nogata Fukuoka,W3
5,"Sumgong, Jemima Jelagat",16.22,32.58,65.83,69.47,82.45,99.33,116.37,132.95,140.68,5.37,28,4,4,29,F,KEN,others,Nandi,F3
6,"Hug, Marcel E.",8.38,17.65,37.65,39.72,47.67,58.60,70.23,79.83,84.65,3.23,4,4,4,28,M,SUI,others,Neuenkirch,W4
7,"Geneti, Markos",15.17,30.48,61.62,64.85,76.95,92.52,107.47,122.80,129.83,4.97,5,5,5,29,M,ETH,others,Addis Ababa,5


In [4]:
df.dtypes

name                    object
5k                     float64
10k                    float64
20k                    float64
half                   float64
25k                    float64
30k                    float64
35k                    float64
40k                    float64
official               float64
pace                   float64
rank_overall             int64
rank_in_gender_only      int64
rank_in_division         int64
age                      int64
gender                  object
country                 object
state                   object
city                    object
bib                     object
dtype: object

## Drop 'name' & 'bib'

These values to identiy runners. In data, we identify them by rows only. So, drop them.

In [5]:
df.drop(['name', 'bib'], axis=1, inplace=True)

from pandas_profiling import ProfileReport
profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)
profile.to_file("0_PROFILLING_2014.html")

## Drop 'city'

In [6]:
df.drop(['city'], axis=1, inplace=True)

## Feature Engineering

### New Feature -- AgeGroup

In [7]:
# Define age_division
def age_to_age_division(age):
    if age<=34            : return 0
    if (35<=age)&(age<=39) : return 1
    if (40<=age)&(age<=44) : return 2
    if (45<=age)&(age<=49) : return 3
    if (50<=age)&(age<=54) : return 4
    if (55<=age)&(age<=59) : return 5
    if (60<=age)&(age<=64) : return 6
    if (65<=age)&(age<=69) : return 7
    if (70<=age)&(age<=74) : return 8
    if (75<=age)&(age<=79) : return 9
    if age>=80             : return 10
    
age_division = df['age'].apply(age_to_age_division)

df.insert(14, 'age_division', value=age_division)

In [8]:
df[['age', 'age_division']].head(8)

,age,age_division
0,47,3
1,33,0
2,41,2
3,24,0
4,40,2
5,29,0
6,28,0
7,29,0


### New Features -- avgSpeed*

In [9]:
milestones = [['5k', '10k', '20k', '25k', '30k', '35k', '40k'],
              [ 5,    10,    20,    25,    30,    35,    40]]
loc = 10
for i, j in zip(*milestones):
    values = df[i] / j
    df.insert(loc, i+'_avgSpeed', values)
    loc += 1

In [10]:
df.head(8)

,5k,10k,20k,half,25k,30k,35k,40k,official,pace,5k_avgSpeed,10k_avgSpeed,20k_avgSpeed,25k_avgSpeed,30k_avgSpeed,35k_avgSpeed,40k_avgSpeed,rank_overall,rank_in_gender_only,rank_in_division,age,age_division,gender,country,state
0,8.02,17.37,37.65,39.72,47.67,59.18,71.40,80.43,85.25,3.27,1.604,1.737,1.8825,1.9068,1.972667,2.040000,2.01075,8,8,8,47,3,M,JPN,others
1,16.22,32.58,65.83,69.47,82.43,99.33,116.37,132.10,138.95,5.30,3.244,3.258,3.2915,3.2972,3.311000,3.324857,3.30250,21,1,1,33,0,F,KEN,others
2,7.75,16.62,36.10,38.03,45.80,56.45,67.42,76.10,80.60,3.08,1.550,1.662,1.8050,1.8320,1.881667,1.926286,1.90250,1,1,1,41,2,M,RSA,others
3,16.20,32.57,65.83,69.47,82.43,99.33,116.37,132.95,140.58,5.37,3.240,3.257,3.2915,3.2972,3.311000,3.324857,3.32375,27,3,3,24,0,F,ETH,others
4,8.02,17.12,36.58,38.60,46.37,57.03,67.83,76.72,81.23,3.10,1.604,1.712,1.8290,1.8548,1.901000,1.938000,1.91800,2,2,2,40,2,M,JPN,others
5,16.22,32.58,65.83,69.47,82.45,99.33,116.37,132.95,140.68,5.37,3.244,3.258,3.2915,3.2980,3.311000,3.324857,3.32375,28,4,4,29,0,F,KEN,others
6,8.38,17.65,37.65,39.72,47.67,58.60,70.23,79.83,84.65,3.23,1.676,1.765,1.8825,1.9068,1.953333,2.006571,1.99575,4,4,4,28,0,M,SUI,others
7,15.17,30.48,61.62,64.85,76.95,92.52,107.47,122.80,129.83,4.97,3.034,3.048,3.0810,3.0780,3.084000,3.070571,3.07000,5,5,5,29,0,M,ETH,others


In [11]:
milestones = []
for i in range(5, 45, 5) : milestones.append(i)
print(milestones)

[5, 10, 15, 20, 25, 30, 35, 40]


### Standard Scaling of Numericals

As clustering algorithms are distance-based, so differences of units have a highly affects on algorithms performance. We need to scale numerical columns to a range of 0 to 1.

In [12]:
#to_scale = ['5k', '10k', '20k', 'half', '25k', '30k', '35k', '40k', 'official', 'pace', 'age']

to_scale = list(df.select_dtypes(exclude='O'))
scaler = MinMaxScaler()

#df = df.copy()

df[to_scale] = scaler.fit_transform(df[to_scale])

In [13]:
df.head()

,5k,10k,20k,half,25k,30k,35k,40k,official,pace,5k_avgSpeed,10k_avgSpeed,20k_avgSpeed,25k_avgSpeed,30k_avgSpeed,35k_avgSpeed,40k_avgSpeed,rank_overall,rank_in_gender_only,rank_in_division,age,age_division,gender,country,state
0,0.003409,0.007832,0.008234,0.008508,0.007689,0.008533,0.010421,0.010017,0.010147,0.010863,0.003409,0.007832,0.008234,0.007689,0.008533,0.010421,0.010017,0.000219,0.000398,0.001003,0.460317,0.3,M,JPN,others
1,0.106944,0.166667,0.157928,0.158276,0.150604,0.134029,0.128172,0.129555,0.127324,0.126930,0.106944,0.166667,0.157928,0.150604,0.134029,0.128172,0.129555,0.000626,0.000000,0.000000,0.238095,0.0,F,KEN,others
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365079,0.2,M,RSA,others
3,0.106692,0.166562,0.157928,0.158276,0.150604,0.134029,0.128172,0.131521,0.130881,0.130932,0.106692,0.166562,0.157928,0.150604,0.134029,0.128172,0.131521,0.000814,0.000114,0.000287,0.095238,0.0,F,ETH,others
4,0.003409,0.005221,0.002550,0.002870,0.002344,0.001813,0.001074,0.001434,0.001375,0.001144,0.003409,0.005221,0.002550,0.002344,0.001813,0.001074,0.001434,0.000031,0.000057,0.000143,0.349206,0.2,M,JPN,others


### Version 1: Non-onehot for KPrototypes

In [14]:
df.to_csv('cleanData_to_kPrototypes.csv', index=False)

In [15]:
df.head(8)

,5k,10k,20k,half,25k,30k,35k,40k,official,pace,5k_avgSpeed,10k_avgSpeed,20k_avgSpeed,25k_avgSpeed,30k_avgSpeed,35k_avgSpeed,40k_avgSpeed,rank_overall,rank_in_gender_only,rank_in_division,age,age_division,gender,country,state
0,0.003409,0.007832,0.008234,0.008508,0.007689,0.008533,0.010421,0.010017,0.010147,0.010863,0.003409,0.007832,0.008234,0.007689,0.008533,0.010421,0.010017,0.000219,0.000398,0.001003,0.460317,0.3,M,JPN,others
1,0.106944,0.166667,0.157928,0.158276,0.150604,0.134029,0.128172,0.129555,0.127324,0.126930,0.106944,0.166667,0.157928,0.150604,0.134029,0.128172,0.129555,0.000626,0.000000,0.000000,0.238095,0.0,F,KEN,others
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365079,0.2,M,RSA,others
3,0.106692,0.166562,0.157928,0.158276,0.150604,0.134029,0.128172,0.131521,0.130881,0.130932,0.106692,0.166562,0.157928,0.150604,0.134029,0.128172,0.131521,0.000814,0.000114,0.000287,0.095238,0.0,F,ETH,others
4,0.003409,0.005221,0.002550,0.002870,0.002344,0.001813,0.001074,0.001434,0.001375,0.001144,0.003409,0.005221,0.002550,0.002344,0.001813,0.001074,0.001434,0.000031,0.000057,0.000143,0.349206,0.2,M,JPN,others
5,0.106944,0.166667,0.157928,0.158276,0.150687,0.134029,0.128172,0.131521,0.131099,0.130932,0.106944,0.166667,0.157928,0.150687,0.134029,0.128172,0.131521,0.000846,0.000171,0.000430,0.174603,0.0,F,KEN,others
6,0.007955,0.010756,0.008234,0.008508,0.007689,0.006720,0.007358,0.008629,0.008837,0.008576,0.007955,0.010756,0.008234,0.007689,0.006720,0.007358,0.008629,0.000094,0.000171,0.000430,0.158730,0.0,M,SUI,others
7,0.093687,0.144737,0.135564,0.135018,0.128073,0.112743,0.104868,0.108039,0.107423,0.108062,0.093687,0.144737,0.135564,0.128073,0.112743,0.104868,0.108039,0.000125,0.000228,0.000573,0.174603,0.0,M,ETH,others


### Version 2: Onehot Encoding for K-means Clustering

In [16]:
df_kmeans = df.copy()

In [17]:
to_onehot = ['age_division', 'gender', 
             'country', 'state']

# to_onehot not includes 'city' as it has around 6000 uniques, which could damages the clustering model.

df_kmeans = pd.get_dummies(df, columns=to_onehot)

In [18]:
print('Number of columns: ', len(df_kmeans.columns))

Number of columns:  181


In [19]:
df_kmeans.to_csv('cleanData_to_kMeans.csv', index=False)

In [20]:
df_kmeans.head()

,5k,10k,20k,half,25k,30k,35k,40k,official,pace,5k_avgSpeed,10k_avgSpeed,20k_avgSpeed,25k_avgSpeed,30k_avgSpeed,35k_avgSpeed,40k_avgSpeed,rank_overall,rank_in_gender_only,rank_in_division,age,age_division_0.0,age_division_0.1,age_division_0.2,age_division_0.30000000000000004,age_division_0.4,age_division_0.5,age_division_0.6000000000000001,age_division_0.7000000000000001,age_division_0.8,age_division_0.9,age_division_1.0,gender_F,gender_M,country_ARG,country_AUS,country_AUT,country_BAH,country_BAR,country_BEL,country_BER,country_BLR,country_BOL,country_BRA,country_BRN,country_CAN,country_CAY,country_CHI,country_CHN,country_COL,country_CRC,country_CRO,country_CZE,country_DEN,country_DOM,country_ECU,country_EGY,country_ESA,country_ESP,country_EST,country_ETH,country_FIN,country_FRA,country_GBR,country_GER,country_GRE,country_GUA,country_HKG,country_HUN,country_INA,country_IND,country_IRL,country_ISL,country_ISR,country_ITA,country_JPN,country_KEN,country_KOR,country_LIE,country_LUX,country_MAR,country_MAS,country_MEX,country_NED,country_NOR,country_NZL,country_PAN,country_PER,country_PHI,country_POL,country_POR,country_QAT,country_ROU,country_RSA,country_RUS,country_SIN,country_SLO,country_SUI,country_SVK,country_SWE,country_TAN,country_THA,country_TPE,country_TRI,country_TUR,country_UAE,country_UKR,country_URU,country_USA,country_VEN,country_VGB,country_ZIM,state_AA,state_AB,state_AE,state_AK,state_AL,state_AP,state_AR,state_AZ,state_BC,state_CA,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,state_GU,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MB,state_MD,state_ME,state_MI,state_MN,state_MO,state_MS,state_MT,state_NB,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NL,state_NM,state_NS,state_NT,state_NV,state_NY,state_OH,state_OK,state_ON,state_OR,state_PA,state_PE,state_PR,state_QC,state_RI,state_SC,state_SD,state_SK,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,state_YT,state_others
0,0.003409,0.007832,0.008234,0.008508,0.007689,0.008533,0.010421,0.010017,0.010147,0.010863,0.003409,0.007832,0.008234,0.007689,0.008533,0.010421,0.010017,0.000219,0.000398,0.001003,0.460317,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.106944,0.166667,0.157928,0.158276,0.150604,0.134029,0.128172,0.129555,0.127324,0.126930,0.106944,0.166667,0.157928,0.150604,0.134029,0.128172,0.129555,0.000626,0.000000,0.000000,0.238095,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365079,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.106692,0.166562,0.157928,0.158276,0.150604,0.134029,0.128172,0.131521,0.130881,0.130932,0.106692,0.166562,0.157928,0.150604,0.134029,0.128172,0.131521,0.000814,0.000114,0.000287,0.095238,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0